# **Worksapce set up**

In [15]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import tempfile
import zipfile
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

Populating the interactive namespace from numpy and matplotlib


In [17]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [18]:
# Loads the data and splits it into 60% training and 40% testing sets
df = loadtxt("drive/MyDrive/GE_practicum/diabetes.csv", delimiter=",", skiprows=1)

x_train = df[0:int(0.6*len(df)),0:8]
y_train = df[0:int(0.6*len(df)),8]

x_test = df[int(0.6*len(df)):,0:8]
y_test = df[int(0.6*len(df)):,8]

# **Clustering**

In [19]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/ANN_base')
base_model.summary()

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(base_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               1152      
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 11,521
Trainable params: 11,521
Non-trainable params: 0
_________________________________________________________________
Saved baseline model to: /tmp/tmp6uztsafe.h5


In [20]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

batch_size = 10
epochs = 10

num_images = x_train.shape[0]
end_step = np.ceil(num_images / batch_size).astype(np.int8) * epochs

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

pruned_model = prune_low_magnitude(base_model, **pruning_params)

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
pruned_model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])
pruned_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense_8  (None, 128)              2178      
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_9  (None, 64)               16450     
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_1  (None, 32)               4130      
 0 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_1  (None, 1)                67        
 1 (PruneLowMagnitude)                                           
                                                                 
Total params: 22,825
Trainable params: 11,521
Non-trai

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)


In [21]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
]

pruned_model.fit(x_train,y_train,epochs = epochs,validation_data = (x_test,y_test), batch_size=batch_size, callbacks=callbacks)
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(pruned_model, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Epoch 1/10
46/46 [==============================] - 4s 19ms/step - loss: 0.4759 - accuracy: 0.7826 - val_loss: 0.5991 - val_accuracy: 0.7500
Epoch 2/10
46/46 [==============================] - 1s 11ms/step - loss: 0.4661 - accuracy: 0.7804 - val_loss: 0.5903 - val_accuracy: 0.7695
Epoch 3/10
46/46 [==============================] - 0s 7ms/step - loss: 0.7480 - accuracy: 0.4826 - val_loss: 0.8328 - val_accuracy: 0.3734
Epoch 4/10
46/46 [==============================] - 0s 8ms/step - loss: 0.7907 - accuracy: 0.4239 - val_loss: 0.8200 - val_accuracy: 0.3734
Epoch 5/10
46/46 [==============================] - 0s 7ms/step - loss: 0.9165 - accuracy: 0.5761 - val_loss: 0.9112 - val_accuracy: 0.6981
Epoch 6/10
46/46 [==============================] - 0s 9ms/step - loss: 1.0178 - accuracy: 0.6174 - val_loss: 0.8913 - val_accuracy: 0.6981
Epoch 7/10
46/46 [==============================] - 0s 9ms/step - loss: 1.0320 - accuracy: 0.6196 - val_loss: 0.9113 - val_accuracy: 0.6981
Epoch 8/10
46/46 [

Saved pruned Keras model to: /tmp/tmpra2yfyt1.h5


# **Quantization and conversion to tflite**

In [22]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('ANN_pruned.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmprodw95_p/assets


INFO:tensorflow:Assets written to: /tmp/tmprodw95_p/assets


14768

In [23]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the FULLY_CONNECTED op takes
tensor #0 and tensor #5 and tensor #2 as input and produces tensor #9 as output.

Subgraph#0 main(T#0) -> [T#13]
  Op#0 FULLY_CONNECTED(T#0, T#5, T#2) -> [T#9]
  Op#1 FULLY_CONNECTED(T#9, T#6, T#1) -> [T#10]
  Op#2 FULLY_CONNECTED(T#10, T#7, T#4) -> [T#11]
  Op#3 FULLY_CONNECTED(T#11, T#8, T#3) -> [T#12]
  Op#4 LOGISTIC(T#12) -> [T#13]

Tensors of Subgraph#0
  T#0(serving_default_dense_8_input:0) shape_signature:[-1, 8], type:FLOAT32
  T#1(dense_9/bias) shape:[64], type:FLOAT32 RO 256 bytes
  T#2(dense_8/bias) shape:[128], type:FLOAT32 RO 512 bytes
  T#3(dense_11/bias) shape:[1], type:FLOAT32 RO 4 bytes
  T#4(dense_10/bias) shape:[32], type:FLOAT32 RO 128 bytes
  T#5(sequential_2/dense_8/MatMul) shape:[128, 8], type:INT8 RO 1024 bytes
  T#6(sequential_2/dense_9/MatMul) shape:[64, 128], type:INT

In [24]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(int(result[0]*2))

right_pred = [y_test[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.698051948051948


# **Conversion to C array**

In [25]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [26]:
# Write TFLite model to a C source (or header) file
with open("ANN_pruned" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "ANN_pruned"))

# **Size Comparison**

In [27]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [28]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size('ANN_pruned.tflite')))

Size of gzipped baseline Keras model: 45026.00 bytes
Size of gzipped pruned Keras model: 16165.00 bytes
Size of gzipped pruned TFlite model: 5883.00 bytes
